In [2]:
def leap_year(year):
    if year % 4 == 0 and year %100 != 0 or year % 400 == 0:
        return True
    else:
        return False

In [3]:
def mean_0(time_0,x,y,n,set_point,lon_input,lat_input,time_input,temp_input):# n is the periods in a day.
    import numpy as np
    long = lon_input #Defining the variables in the netcdf file and assigning them 
    lats = lat_input
    time = temp_input
    x_new=((x%time_0)*(n*365)) 
    y_new=((y%time_0)*(n*365))
    number_of_years=y-x
    time_period=time[x_new:y_new] 
    temperature = temp_input[x_new:y_new][:][:]
    temperature_new= np.empty((len(time_period)/n, len(lats),len(long)), dtype=np.float32)
    for i in range(len(temperature_new)):   
        temperature_new[i]= (np.sum(temperature[(i*n):((i+1)*n)],axis=0))/4
    temperature_mean=np.empty((365,len(lats),len(long)), dtype=np.float32)
    for i in range(365):
        for j in range(number_of_years):
            temperature_mean[i]= temperature_new[i+365*j]+temperature_mean[i]
    temperature_mean=temperature_mean/number_of_years
    temperature_mean = np.subtract(temperature_mean,(273.16+set_point))
    time_for_mean=np.arange(1,366,1)
    return (temperature_mean)

In [4]:
def max_0(time_0,x,y,n,set_point,lon_input,lat_input,time_input,temp_input):
    import numpy as np
    long = lon_input
    lats = lat_input
    time = temp_input
    x_new=((x%time_0)*(n*365)) 
    y_new=((y%time_0)*(n*365))
    number_of_years=y-x
    time_period=time[x_new:y_new] 
    temperature = temp_input[x_new:y_new][:][:]
    temperature_new= np.empty((len(time_period)/n, len(lats),len(long)), dtype=np.float32)
    for i in range(len(temperature_new)):   
        temperature_new[i]= np.max(temperature[(i*n):((i+1)*n)],axis=0)
    temperature_max=np.empty((365,len(lats),len(long)), dtype=np.float32)
    for i in range(365):
        for j in range(number_of_years):
            temperature_max[i]= temperature_new[i+365*j]+temperature_max[i]
    temperature_max=temperature_max/number_of_years
    temperature_max = np.subtract(temperature_max,(273.16+set_point))
    time_for_max=np.arange(1,366,1)
    return (temperature_max)

In [5]:
def min_0(time_0,x,y,n,set_point,lon_input,lat_input,time_input,temp_input):# n is the periods in a day.
    import numpy as np
    long = lon_input
    lats = lat_input
    time = temp_input
    x_new=((x%time_0)*(n*365)) 
    y_new=((y%time_0)*(n*365))
    number_of_years=y-x
    time_period=time[x_new:y_new] 
    temperature = temp_input[x_new:y_new][:][:]
    temperature_new= np.empty((len(time_period)/n, len(lats),len(long)), dtype=np.float32)
    for i in range(len(temperature_new)):   
        temperature_new[i]= np.min(temperature[(i*n):((i+1)*n)],axis=0)
    temperature_min=np.empty((365,len(lats),len(long)), dtype=np.float32)
    for i in range(365):
        for j in range(number_of_years):
            temperature_min[i]= temperature_new[i+365*j]+temperature_min[i]
    temperature_min=temperature_min/number_of_years
    temperature_min = np.subtract(temperature_min,(273.16+set_point))
    time_for_min=np.arange(1,366,1)
    return (temperature_min)

In [6]:
def two_point_average(time_0,x,y,n,set_point,lon_input,lat_input,time_input,temp_input):
    import numpy as np
    long = lon_input
    lats = lat_input
    time = temp_input
    x_new=((x%time_0)*(n*365)) 
    y_new=((y%time_0)*(n*365))
    number_of_years=y-x
    time_period=time[x_new:y_new] 
    temperature = temp_input[x_new:y_new][:][:]
    temperature_new= np.empty((len(time_period)/n, len(lats),len(long)), dtype=np.float32)
    for i in range(len(temperature_new)):   
        temperature_new[i]= (np.min(temperature[(i*n):((i+1)*n)],axis=0)+np.max(temperature[(i*n):((i+1)*n)],axis=0))/2
    temperature_2_point=np.empty((365,len(lats),len(long)), dtype=np.float32)
    for i in range(365):
        for j in range(number_of_years):
            temperature_2_point[i]= temperature_new[i+365*j]+temperature_2_point[i]
    temperature_2_point=temperature_2_point/number_of_years
    temperature_2_point = np.subtract(temperature_2_point,(273.16+set_point))
    time_for_min=np.arange(1,366,1)
    return (temperature_2_point)

In [ ]:
def new_HDD(temp_0,x,y,n,set_point):
    import numpy as np
    from netCDF4 import Dataset
    nc=Dataset('/Users/Bora/Desktop/Bond/HDD_20/01.nc','r')
    for i in nc.variables:
        print ([i,nc.variables[i].units,nc.variables[i].shape])
    lons_0 = np.array(nc.variables['longitude'][:],dtype=np.float32)
    lats_0 = np.array(nc.variables ['latitude'][:],dtype=np.float32)
    time_0 = np.array(nc.variables ['time'][:],dtype=np.float32)
    print("About to create temp_0_part1")
    slice_int=int(len(time_0)/2)
    temperature_0_part1 = np.array(nc.variables['t2m'][:slice_int],dtype=np.float16)
    print("About to create temp_0_part2")
    temperature_0_part2 = np.array(nc.variables['t2m'][slice_int:],dtype=np.float16)
    print("Initialized arrays")
    temperature_0=np.vstack((temperature_0_part1,temperature_0_part2))
    time_new=np.empty([43800],dtype=np.float32)
    temperature_new = np.empty([43800,241,480],dtype=np.float16)
    nc.close()
    print("Initialized temperature_new & time_new")
    leap_counter = 0
    holder=0
    for i in range(x,y):
        holder = (i-temp_0)*365*4
        feb_0=((i-temp_0)*365+31+28+leap_counter)*24/6
        feb_1= ((i-temp_0)*365+31+28)*24/6
        if ( leap_year(i)):
            #copy all the dates except for feb 29
            temperature_new[holder :feb_1]=temperature_0[(holder+leap_counter*4):feb_0]
            time_new[holder :feb_1]=time_0[(holder+leap_counter*4):feb_0]
            holder=holder+365*4
            leap_counter=leap_counter+1
            temperature_new[feb_1: holder]=temperature_0[(feb_0+4):(holder+4*leap_counter)]
            time_new[feb_1: holder]=time_0[(feb_0+4):(holder+4*leap_counter)]
        else:
            #copy whole year
            temperature_new[holder :(holder+365*4)]=temperature_0[(holder+leap_counter*4):(holder+leap_counter*4+365*4)]
            time_new[holder :(holder+365*4)]=time_0[(holder+leap_counter*4):(holder+leap_counter*4+365*4)]
            holder=holder+365*4
    print("Finished extracting Feb 29")
    mean_temp = mean_0(temp_0,x,y,n,set_point,lons_0,lats_0,time_new,temperature_new)
    max_temp = max_0(temp_0,x,y,n,set_point,lons_0,lats_0,time_new,temperature_new)
    min_temp = min_0(temp_0,x,y,n,set_point,lons_0,lats_0,time_new,temperature_new)
    point_2_temp= two_point_average(temp_0,x,y,n,set_point,lons_0,lats_0,time_new,temperature_new)
    print("Finished different calculation methods")
    time= np.arange(1,366,1)
    lats=lats_0
    long=lons_0
    for i in range (len(time)):
       for j in range (len(lats)):
           for k in range (len(long)):
               if (mean_temp[i][j][k]>0.0):
                   mean_temp[i][j][k] =0.0
               if (max_temp[i][j][k]>0.0):
                   max_temp[i][j][k] =0.0
               if (min_temp[i][j][k]>0.0):
                   min_temp[i][j][k] =0.0
               if (point_2_temp[i][j][k]>0.0):
                   point_2_temp[i][j][k] =0.0
    mean_t2m= np.empty((len(lats),len(long)), dtype=np.float32)
    max_t2m= np.empty((len(lats),len(long)), dtype=np.float32)
    min_t2m= np.empty((len(lats),len(long)), dtype=np.float32)
    point_2_t2m=np.empty((len(lats),len(long)), dtype=np.float32)
    for i in range (365):
        mean_t2m = mean_t2m+mean_temp[i]
        max_t2m = max_t2m+max_temp[i]
        min_t2m = min_t2m+min_temp[i]
        point_2_t2m=point_2_t2m+point_2_temp[i]
    print("Finished analyses, extracting data.")
    year_number=y-x
    cmplgrp = Dataset('/Users/Bora/Desktop/Bond/HDD_'+str(year_number)+'/HDD_'+str(year_number)+'_'+str(set_point)+'.nc', mode='w', format='NETCDF3_CLASSIC')
    cmplgrp.close()
    
    cmplgrp = Dataset('/Users/Bora/Desktop/Bond/HDD_'+str(year_number)+'/HDD_'+str(year_number)+'_'+str(set_point)+'.nc','a')
    
    lat= cmplgrp.createDimension('latitude', 241)
    lon= cmplgrp.createDimension('longitude', 480)
    
    latitudes= cmplgrp.createVariable('latitude',np.float32,('latitude',))
    longitudes=cmplgrp.createVariable('longitude',np.float32,('longitude',))
    temp_mean= cmplgrp.createVariable('t2m_mean',np.short,('latitude','longitude',))
    temp_min= cmplgrp.createVariable('t2m_min',np.short,('latitude','longitude',))
    temp_max= cmplgrp.createVariable('t2m_max',np.short,('latitude','longitude',))
    temp_2point=cmplgrp.createVariable('t2m_2point',np.short,('latitude','longitude',))
    
    import time
    cmplgrp.history = 'Created ' + time.ctime(time.time())
    cmplgrp.Conventions = 'CF-1.6'
    
    latitudes.long_name = 'latitude'
    latitudes.units = 'degrees_north'
    
    longitudes.units = 'degrees_east'
    longitudes.long_name='longitude'
    
    temp_mean.long_name = '2 meter mean temperature averages for 20 years'
    temp_mean.units = 'K'
    temp_mean.scale_factor = 1
    temp_mean.add_offset=0
    temp_mean.fill_value=-32767
    temp_mean.missing_value = -32767
    
    temp_min.long_name = '2 meter min temperature averages for 20 years'
    temp_min.units = 'K'
    temp_min.scale_factor = 1
    temp_min.add_offset=0
    temp_min.fill_value=-32767
    temp_min.missing_value = -32767
    
    temp_max.long_name = '2 meter max temperature averages for 20 years'
    temp_max.units = 'K'
    temp_max.scale_factor = 1
    temp_max.add_offset=0
    temp_max.fill_value=-32767
    temp_max.missing_value = -32767
    
    temp_2point.long_name = '2 meter max temperature averages for 20 years'
    temp_2point.units = 'K'
    temp_2point.scale_factor = 1
    temp_2point.add_offset=0
    temp_2point.fill_value=-32767
    temp_2point.missing_value = -32767
    
    latitudes[:]= lats
    longitudes[:]=long
    temp_mean[:]=mean_t2m
    temp_min[:]=min_t2m
    temp_max[:]=max_t2m
    temp_2point[:]=point_2_t2m
    
    cmplgrp.close()

In [ ]:
new_HDD(1996,1996,2016,4,18)

['longitude', 'degrees_east', (480,)]
['latitude', 'degrees_north', (241,)]
['time', 'hours since 1900-01-01 00:00:0.0', (29220,)]
['t2m', 'K', (29220, 241, 480)]
About to create temp_0_part1
About to create temp_0_part2
Initialized arrays
Initialized temperature_new & time_new


/Users/Bora/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:29: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/Bora/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:30: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/Bora/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:33: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/Bora/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:34: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


Finished extracting Feb 29


/Users/Bora/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:11: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


Finished different calculation methods
Finished analyses, extracting data.


In [ ]:
cmplgrp.close()